In [36]:
import subprocess
import json
from tqdm import tqdm
from multiprocessing import Pool

In [31]:
resources = "AWS::EC2::CustomerGateway | AWS::EC2::EIP | AWS::EC2::Host | AWS::EC2::Instance | AWS::EC2::InternetGateway | AWS::EC2::NetworkAcl | AWS::EC2::NetworkInterface | AWS::EC2::RouteTable | AWS::EC2::SecurityGroup | AWS::EC2::Subnet | AWS::CloudTrail::Trail | AWS::EC2::Volume | AWS::EC2::VPC | AWS::EC2::VPNConnection | AWS::EC2::VPNGateway | AWS::EC2::RegisteredHAInstance | AWS::EC2::NatGateway | AWS::EC2::EgressOnlyInternetGateway | AWS::EC2::VPCEndpoint | AWS::EC2::VPCEndpointService | AWS::EC2::FlowLog | AWS::EC2::VPCPeeringConnection | AWS::Elasticsearch::Domain | AWS::IAM::Group | AWS::IAM::Policy | AWS::IAM::Role | AWS::IAM::User | AWS::ElasticLoadBalancingV2::LoadBalancer | AWS::ACM::Certificate | AWS::RDS::DBInstance | AWS::RDS::DBSubnetGroup | AWS::RDS::DBSecurityGroup | AWS::RDS::DBSnapshot | AWS::RDS::DBCluster | AWS::RDS::DBClusterSnapshot | AWS::RDS::EventSubscription | AWS::S3::Bucket | AWS::S3::AccountPublicAccessBlock | AWS::Redshift::Cluster | AWS::Redshift::ClusterSnapshot | AWS::Redshift::ClusterParameterGroup | AWS::Redshift::ClusterSecurityGroup | AWS::Redshift::ClusterSubnetGroup | AWS::Redshift::EventSubscription | AWS::SSM::ManagedInstanceInventory | AWS::CloudWatch::Alarm | AWS::CloudFormation::Stack | AWS::ElasticLoadBalancing::LoadBalancer | AWS::AutoScaling::AutoScalingGroup | AWS::AutoScaling::LaunchConfiguration | AWS::AutoScaling::ScalingPolicy | AWS::AutoScaling::ScheduledAction | AWS::DynamoDB::Table | AWS::CodeBuild::Project | AWS::WAF::RateBasedRule | AWS::WAF::Rule | AWS::WAF::RuleGroup | AWS::WAF::WebACL | AWS::WAFRegional::RateBasedRule | AWS::WAFRegional::Rule | AWS::WAFRegional::RuleGroup | AWS::WAFRegional::WebACL | AWS::CloudFront::Distribution | AWS::CloudFront::StreamingDistribution | AWS::Lambda::Function | AWS::NetworkFirewall::Firewall | AWS::NetworkFirewall::FirewallPolicy | AWS::NetworkFirewall::RuleGroup | AWS::ElasticBeanstalk::Application | AWS::ElasticBeanstalk::ApplicationVersion | AWS::ElasticBeanstalk::Environment | AWS::WAFv2::WebACL | AWS::WAFv2::RuleGroup | AWS::WAFv2::IPSet | AWS::WAFv2::RegexPatternSet | AWS::WAFv2::ManagedRuleSet | AWS::XRay::EncryptionConfig | AWS::SSM::AssociationCompliance | AWS::SSM::PatchCompliance | AWS::Shield::Protection | AWS::ShieldRegional::Protection | AWS::Config::ConformancePackCompliance | AWS::Config::ResourceCompliance | AWS::ApiGateway::Stage | AWS::ApiGateway::RestApi | AWS::ApiGatewayV2::Stage | AWS::ApiGatewayV2::Api | AWS::CodePipeline::Pipeline | AWS::ServiceCatalog::CloudFormationProvisionedProduct | AWS::ServiceCatalog::CloudFormationProduct | AWS::ServiceCatalog::Portfolio | AWS::SQS::Queue | AWS::KMS::Key | AWS::QLDB::Ledger | AWS::SecretsManager::Secret | AWS::SNS::Topic | AWS::SSM::FileData | AWS::Backup::BackupPlan | AWS::Backup::BackupSelection | AWS::Backup::BackupVault | AWS::Backup::RecoveryPoint | AWS::ECR::Repository | AWS::ECS::Cluster | AWS::ECS::Service | AWS::ECS::TaskDefinition | AWS::EFS::AccessPoint | AWS::EFS::FileSystem | AWS::EKS::Cluster | AWS::OpenSearch::Domain | AWS::EC2::TransitGateway | AWS::Kinesis::Stream | AWS::Kinesis::StreamConsumer | AWS::CodeDeploy::Application | AWS::CodeDeploy::DeploymentConfig | AWS::CodeDeploy::DeploymentGroup | AWS::EC2::LaunchTemplate | AWS::ECR::PublicRepository | AWS::GuardDuty::Detector | AWS::EMR::SecurityConfiguration | AWS::SageMaker::CodeRepository | AWS::Route53Resolver::ResolverEndpoint | AWS::Route53Resolver::ResolverRule | AWS::Route53Resolver::ResolverRuleAssociation | AWS::DMS::ReplicationSubnetGroup | AWS::DMS::EventSubscription | AWS::MSK::Cluster | AWS::StepFunctions::Activity | AWS::WorkSpaces::Workspace | AWS::WorkSpaces::ConnectionAlias | AWS::SageMaker::Model | AWS::ElasticLoadBalancingV2::Listener | AWS::StepFunctions::StateMachine | AWS::Batch::JobQueue | AWS::Batch::ComputeEnvironment | AWS::AccessAnalyzer::Analyzer | AWS::Athena::WorkGroup | AWS::Athena::DataCatalog | AWS::Detective::Graph | AWS::GlobalAccelerator::Accelerator | AWS::GlobalAccelerator::EndpointGroup | AWS::GlobalAccelerator::Listener | AWS::EC2::TransitGatewayAttachment | AWS::EC2::TransitGatewayRouteTable | AWS::DMS::Certificate | AWS::AppConfig::Application | AWS::AppSync::GraphQLApi | AWS::DataSync::LocationSMB | AWS::DataSync::LocationFSxLustre | AWS::DataSync::LocationS3 | AWS::DataSync::LocationEFS | AWS::DataSync::Task | AWS::DataSync::LocationNFS | AWS::EC2::NetworkInsightsAccessScopeAnalysis | AWS::EKS::FargateProfile | AWS::Glue::Job | AWS::GuardDuty::ThreatIntelSet | AWS::GuardDuty::IPSet | AWS::SageMaker::Workteam | AWS::SageMaker::NotebookInstanceLifecycleConfig | AWS::ServiceDiscovery::Service | AWS::ServiceDiscovery::PublicDnsNamespace | AWS::SES::ContactList | AWS::SES::ConfigurationSet | AWS::Route53::HostedZone"
resource_lists = content_list.split(" | ")

In [26]:
proc = subprocess.Popen(
    ['aws', 'configservice', 'list-discovered-resources', '--resource-type', 'AWS::EC2::VPC'],
    stdout = subprocess.PIPE
    )
out, err = proc.communicate()
result = out.decode('utf-8')
result = json.loads(result)

In [27]:
result

{'resourceIdentifiers': [{'resourceType': 'AWS::EC2::VPC',
   'resourceId': 'vpc-03e53262faff38c42'}]}

In [25]:
result['resourceIdentifiers'][0]['resourceId']

'i-0303414e8cd5865c0'

In [39]:
def find_resource(resource_lists) -> list:
    exist_resources = []
    
    for resource in tqdm(resource_lists):
        proc = subprocess.Popen(
            ['aws', 'configservice', 'list-discovered-resources', '--resource-type', f'{resource}'],
        stdout = subprocess.PIPE
        )
        out, err = proc.communicate()
        if 'resourceType' in str(out):
            exist_resources.append(out)
            
    return exist_resources

In [44]:
exist_resources = []

def find_resource(resource) -> str:    
#     for item in tqdm(resource):
    proc = subprocess.Popen(
        ['aws', 'configservice', 'list-discovered-resources', '--resource-type', f'{item}'],
    stdout = subprocess.PIPE
    )
    out, err = proc.communicate()
    
    if 'resourceType' in str(out):
        exist_resources.append(out)


In [45]:
def multiprocessing(resource_lists) -> list:
    resource_lists = resource_lists
    pool = Pool(processes=4)
    pool.map(find_resource, resource_lists)
    pool.close()
    pool.join()

In [ ]:
multiprocessing(resource_lists)